In [43]:
import numpy as np
import pandas as pd
%pip install ucimlrepo
from ucimlrepo import fetch_ucirepo 

Note: you may need to restart the kernel to use updated packages.


In [44]:
individual_household_electric_power_consumption = fetch_ucirepo(id=235) 
  
# data (as pandas dataframes) 
df = individual_household_electric_power_consumption.data.features 
# y = individual_household_electric_power_consumption.data.targets 

df.info()

c:\Users\apost\AppData\Local\Programs\Python\Python313\Lib\site-packages\ucimlrepo\fetch.py:97: DtypeWarning: Columns (2,3,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_url)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075259 entries, 0 to 2075258
Data columns (total 9 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Date                   object 
 1   Time                   object 
 2   Global_active_power    object 
 3   Global_reactive_power  object 
 4   Voltage                object 
 5   Global_intensity       object 
 6   Sub_metering_1         object 
 7   Sub_metering_2         object 
 8   Sub_metering_3         float64
dtypes: float64(1), object(8)
memory usage: 142.5+ MB


## Invalid Readings - Missing Values - Cleanup

In [45]:
total_rows = df.shape[0]
mask = ((df == "?") | (df.isnull()))

missing_rows = mask.any(axis = 1).sum()
missing_columns = mask.any(axis = 0)
missing_columns = missing_columns[missing_columns].index.tolist()

percentage = (missing_rows / total_rows) * 100

print(f"Percentage of missing rows: {percentage:.2f}%")
print(missing_columns)

Percentage of missing rows: 1.25%
['Global_active_power', 'Global_reactive_power', 'Voltage', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']


In [46]:
df.replace("?", np.nan, inplace=True)
temp = df["Date"] + " " + df["Time"]
df["Datetime"] = pd.to_datetime(temp, format="%d/%m/%Y %H:%M:%S")
df.set_index("Datetime", inplace=True)

for column in missing_columns:
    df[column] = df[column].astype(float)
    df[column] = df[column].interpolate(method="time", limit_direction="both")

In [47]:

df[missing_columns].isna().sum()
df.head()

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
Datetime,,,,,,,,,
2006-12-16 17:24:00,16/12/2006,17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
2006-12-16 17:25:00,16/12/2006,17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2006-12-16 17:26:00,16/12/2006,17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
2006-12-16 17:27:00,16/12/2006,17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
2006-12-16 17:28:00,16/12/2006,17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0


## Aggregate Features

In [48]:
daily_df = df.resample("D").agg({
    "Global_active_power": "sum",
    "Global_reactive_power": "sum",
    "Voltage": "mean",
    "Global_intensity": "mean",
    "Sub_metering_1":  "sum",
    "Sub_metering_2":  "sum",
    "Sub_metering_3":  "sum"
})

In [49]:
daily_df["Global_active_power"] = daily_df["Global_active_power"] / 60
daily_df["Sub_metering_1"] = daily_df["Sub_metering_1"] / 1000
daily_df["Sub_metering_2"] = daily_df["Sub_metering_2"] / 1000
daily_df["Sub_metering_3"] = daily_df["Sub_metering_3"] / 1000

In [50]:
peak_hour_power = df["Global_active_power"].resample("D").max()
daily_df["peak_hour_power"] = peak_hour_power

In [51]:
night_mask = (df.index.hour >= 22) | (df.index.hour < 6)
shifted_index = df.index.copy()
shifted_index = shifted_index - pd.to_timedelta((shifted_index.hour < 6).astype(int), unit="D")
night_time_usage = df.loc[night_mask, "Global_active_power"].groupby(shifted_index[night_mask].normalize()).sum()
night_time_usage = night_time_usage.reindex(daily_df.index, fill_value=0)
daily_df["night_time_usage"] = night_time_usage

In [52]:
daily_df["daily_weekend_usage"] = (df["Global_active_power"].where(df.index.day_of_week >= 5, 0).resample("D").sum())

## Categorical Features

In [53]:
daily_df["Day"] = daily_df.index.day_name()
daily_df = pd.get_dummies(daily_df, columns=["Day"])
daily_df.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,peak_hour_power,night_time_usage,daily_weekend_usage,Day_Friday,Day_Monday,Day_Saturday,Day_Sunday,Day_Thursday,Day_Tuesday,Day_Wednesday
Datetime,,,,,,,,,,,,,,,,,
2006-12-16,20.152933,34.922,236.243763,13.082828,0.000,0.546,4.926,7.706,1017.334,1209.176,False,False,True,False,False,False,False
2006-12-17,56.507667,226.006,240.087028,9.999028,2.033,4.187,13.341,7.064,267.578,3390.460,False,False,False,True,False,False,False
2006-12-18,36.730433,161.792,241.231694,6.421667,1.063,2.621,14.018,6.158,344.776,0.000,False,True,False,False,False,False,False
2006-12-19,27.769900,150.942,241.999313,4.926389,0.839,7.602,6.197,7.840,208.098,0.000,False,False,False,False,False,True,False
2006-12-20,37.095800,160.998,242.308062,6.467361,0.000,2.648,14.063,5.988,629.370,0.000,False,False,False,False,False,False,True


In [54]:
seasons = {
    "Winter" : [12, 1, 2],
    "Spring" : [3, 4, 5],
    "Summer" : [6, 7, 8],
    "Autumn" : [9, 10, 11]
}

def get_season(month):
    
    for season, months in seasons.items():
        if month in months:
            return season

daily_df["Season"] = daily_df.index.month.map(get_season)
daily_df = pd.get_dummies(daily_df, columns=["Season"])
daily_df.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,peak_hour_power,night_time_usage,daily_weekend_usage,...,Day_Monday,Day_Saturday,Day_Sunday,Day_Thursday,Day_Tuesday,Day_Wednesday,Season_Autumn,Season_Spring,Season_Summer,Season_Winter
Datetime,,,,,,,,,,,,,,,,,,,,,
2006-12-16,20.152933,34.922,236.243763,13.082828,0.000,0.546,4.926,7.706,1017.334,1209.176,...,False,True,False,False,False,False,False,False,False,True
2006-12-17,56.507667,226.006,240.087028,9.999028,2.033,4.187,13.341,7.064,267.578,3390.460,...,False,False,True,False,False,False,False,False,False,True
2006-12-18,36.730433,161.792,241.231694,6.421667,1.063,2.621,14.018,6.158,344.776,0.000,...,True,False,False,False,False,False,False,False,False,True
2006-12-19,27.769900,150.942,241.999313,4.926389,0.839,7.602,6.197,7.840,208.098,0.000,...,False,False,False,False,True,False,False,False,False,True
2006-12-20,37.095800,160.998,242.308062,6.467361,0.000,2.648,14.063,5.988,629.370,0.000,...,False,False,False,False,False,True,False,False,False,True


In [55]:
daily_df["is_working_Day"] = (~ ((daily_df.index.day_of_week == 5) | (daily_df.index.day_of_week == 6))).astype(int)
daily_df.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,peak_hour_power,night_time_usage,daily_weekend_usage,...,Day_Saturday,Day_Sunday,Day_Thursday,Day_Tuesday,Day_Wednesday,Season_Autumn,Season_Spring,Season_Summer,Season_Winter,is_working_Day
Datetime,,,,,,,,,,,,,,,,,,,,,
2006-12-16,20.152933,34.922,236.243763,13.082828,0.000,0.546,4.926,7.706,1017.334,1209.176,...,True,False,False,False,False,False,False,False,True,0
2006-12-17,56.507667,226.006,240.087028,9.999028,2.033,4.187,13.341,7.064,267.578,3390.460,...,False,True,False,False,False,False,False,False,True,0
2006-12-18,36.730433,161.792,241.231694,6.421667,1.063,2.621,14.018,6.158,344.776,0.000,...,False,False,False,False,False,False,False,False,True,1
2006-12-19,27.769900,150.942,241.999313,4.926389,0.839,7.602,6.197,7.840,208.098,0.000,...,False,False,False,True,False,False,False,False,True,1
2006-12-20,37.095800,160.998,242.308062,6.467361,0.000,2.648,14.063,5.988,629.370,0.000,...,False,False,False,False,True,False,False,False,True,1


In [56]:
daily_df.to_csv("daily_household.csv")
daily_df.isna().sum()

Global_active_power      0
Global_reactive_power    0
Voltage                  0
Global_intensity         0
Sub_metering_1           0
Sub_metering_2           0
Sub_metering_3           0
peak_hour_power          0
night_time_usage         0
daily_weekend_usage      0
Day_Friday               0
Day_Monday               0
Day_Saturday             0
Day_Sunday               0
Day_Thursday             0
Day_Tuesday              0
Day_Wednesday            0
Season_Autumn            0
Season_Spring            0
Season_Summer            0
Season_Winter            0
is_working_Day           0
dtype: int64